DATA SCIENCE PROJECT- Fake News Classification

Author:- Srabosti Dutta

Importing the Dependencies

In [ ]:
import pandas as pd
import numpy as np
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
# Download all the stopwords from the nltk library

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# View the English stopwords that will be unnecessary for our analysis

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Reading the Data

In [ ]:
news_df = pd.read_csv("news1.csv")

In [ ]:
# View the dataframe's first few entries

news_df.head()


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# Number of rows and columns in our dataframe (ROWS, COLUMNS)

news_df.shape

(20800, 5)

In [ ]:
# View the general info about our dataframe columns

news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
# View number of null values in each column

news_df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# Fill our Null values with an empty space

news_df = news_df.fillna('')

Data Preprocessing

In [ ]:
# Merging the title and author columns in our dataframe

news_df['article'] = news_df['title']+news_df['author']

In [ ]:
news_df

,id,title,author,text,label,article
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You FiredWhy the Truth...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Rapper T.I.: Trump a ’Poster Child For White S...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Macy’s Is Said to Receive Takeover Approach by...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
news_df1=news_df.drop('id',axis=1)

In [ ]:
news_df1

,title,author,text,label,article
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You FiredWhy the Truth...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Rapper T.I.: Trump a ’Poster Child For White S...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Macy’s Is Said to Receive Takeover Approach by...
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
news_df1["author"].value_counts()

                                             1957
Pam Key                                       243
admin                                         193
Jerome Hudson                                 166
Charlie Spiering                              141
                                             ... 
Jeremy R. Hammond                               1
Vic Bishop                                      1
Douglas Martin                                  1
Najim Rahim and Fahim Abed                      1
Michael J. de la Merced and Rachel Abrams       1
Name: author, Length: 4202, dtype: int64

In [ ]:
# Separate the Target variable from the dataset

X = news_df1.drop(columns='label', axis=1)
Y = news_df1['label']

In [ ]:
X

,title,author,text,article
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Why the Truth Might Get You FiredWhy the Truth...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Iranian woman jailed for fictional unpublished...
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Rapper T.I.: Trump a ’Poster Child For White S...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Macy’s Is Said to Receive Takeover Approach by...
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
Y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

Stemmming

In [ ]:
p_stemming = PorterStemmer()

In [ ]:
# Function to reduce the word to its original

def stemming(content):
    stemmed_word = re.sub('[^a-zA-Z]',' ',content)
    stemmed_word = stemmed_word.lower()
    stemmed_word = stemmed_word.split()
    stemmed_word = [p_stemming.stem(word) for word in stemmed_word if not word in stopwords.words('english')]
    stemmed_word = ' '.join(stemmed_word)
    return stemmed_word

In [ ]:
news_df1


,title,author,text,label,article
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You FiredWhy the Truth...
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...
...,...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Rapper T.I.: Trump a ’Poster Child For White S...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"N.F.L. Playoffs: Schedule, Matchups and Odds -..."
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Macy’s Is Said to Receive Takeover Approach by...
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
# Apply the above stemming function to the column having the article name and author together. 

news_df1['article'] = news_df1['article'].apply(stemming)

In [ ]:
X = news_df['article'].values
X

array(['House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted ItHouse Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It',
       'FLYNN: Hillary Clinton, Big Woman on Campus - BreitbartFLYNN: Hillary Clinton, Big Woman on Campus - Breitbart',
       'Why the Truth Might Get You FiredWhy the Truth Might Get You Fired',
       ...,
       'Macy’s Is Said to Receive Takeover Approach by Hudson’s Bay - The New York TimesMacy’s Is Said to Receive Takeover Approach by Hudson’s Bay - The New York Times',
       'NATO, Russia To Hold Parallel Exercises In BalkansNATO, Russia To Hold Parallel Exercises In Balkans',
       'What Keeps the F-35 AliveWhat Keeps the F-35 Alive'], dtype=object)

In [ ]:
Y = news_df['label'].values
Y

array([1, 0, 1, ..., 0, 1, 1])

In [ ]:
# Convert the text to numerical data using Vectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
X

<20800x33764 sparse matrix of type '<class 'numpy.float64'>'
	with 249562 stored elements in Compressed Sparse Row format>

In [ ]:
# Splitting of Train and Test data

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 1)

Logistic Regression Model


In [ ]:
# Performing Logistic Regression

ml_model = LogisticRegression()

In [ ]:
# Fit & Train our data

ml_model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
# Check for Accuracy (Train Data)

X_train_predict = ml_model.predict(X_train)
train_data_accuracy = accuracy_score(X_train_predict, Y_train)
percent_tr_accuracy = train_data_accuracy * 100
print("Accuracy for Train data: ", percent_tr_accuracy)

Accuracy for Train data:  94.453125


In [ ]:
# Check for Accuracy (Test Data)

X_test_predict = ml_model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_predict, Y_test)
percent_test_accuracy = test_data_accuracy * 100
print("Accuracy for Test data: ", percent_test_accuracy)

Accuracy for Test data:  92.21153846153847


SUPPORT VECTOR MACHINE


In [ ]:
#Support Vector Machine (SVM)
#Building the model using Support Vector Machine (SVM)
from sklearn.svm import SVC

svc_model = SVC()
svc_model.fit(X_train, Y_train)

SVC()

In [ ]:
# Fit & Train our data

svc_model.fit(X_train, Y_train)

SVC()

In [ ]:
#Prediction from support vector machine model on the testing data
svc_pred = svc_model.predict(X_test)

In [ ]:
# Check for Accuracy (Train Data)

X_train_predict = svc_model.predict(X_train)
train_data_accuracy = accuracy_score(X_train_predict, Y_train)
percent_tr_accuracy = train_data_accuracy * 100
print("Accuracy for Train data: ", percent_tr_accuracy)

Accuracy for Train data:  99.6875


In [ ]:
# Check for Accuracy (Test Data)

X_test_predict = svc_model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_predict, Y_test)
percent_test_accuracy = test_data_accuracy * 100
print("Accuracy for Test data: ", percent_test_accuracy)

Accuracy for Test data:  92.6201923076923


After training the two models
(Logistic Regression Classifier,
Support Vector Machine(SVM) Classifier)
with the training and testing data, we find that SVM is better of the two.

In [ ]:
# Build a Simple Predictive System
#Using the Best of the two models
'''
index = int(input("Enter article number to be verified: "))
^ To get article number as input from user
'''

X_new = X_test[26]
new_predict = svc_model.predict(X_new)
if(new_predict[0]==0):
    print("The News is real")
else:
    print("The News is fake")

The News is fake


Conclusion After using all these news articles, we are able to build a machine learning model (Support Vector Machines-Best of the two) to accurately predict whether or not the news are fake or not.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%%capture
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('Srabosti_Dutta_MLP-B1-5thJuly-PROJECT_FakeNews.ipynb')